# Unsupervised Predict- Movie Recomendation

<img src="https://posteet.com/wp-content/uploads/2019/11/movies.png" width=90%>

# Table of content


- List subheadings we have in this notebook

## 1. Introduction

- Here will intro what the predict is about and our problem statemnt

## 2. Importing liabraries

In [3]:
# utilities
import numpy as np
import pandas as pd

## 3. Uploading the dataset

- here we will upload and subset the dataset givven

## 4. Exploratory Data Analysis

- here we will explore the dataset in details

## 5. Data Preprocessing

- here will do some data cleaning and processing

# Machine Learning

- here will define, resample , fit and train our model

# Conclusion

- we will conclude on our findings

# Submissions

- submissions to be submitted on kaggle